# Analysis of Russian Missile and Drone Attacks against Ukraine

Can we identify potential targets of missile and drone strikes before they land?

This dataset contains available information about launched and shot down missiles and drones during Russian large-scale missile and drone (UAV) strikes on infrastructure (since October 2022) as part of its invasion of Ukraine. The dataset also contains information about some single shot-down UAVs.

The dataset was created manually based on the official reports of Air Force Command of UA Armed Forces and General Staff of the Armed Forces of Ukraine published on social media such as Facebook and Twitter.
You can find a short description of each data feature below:

    time_start - start attack time,
    time_end - end attack time,
    model - missile or UAV type,
    launch_place - city or region from missiles were launched,
    target - could be a city in Ukraine, or region of Ukraine or direction, or full Ukraine,
    carrier - missile launch platform,
    launched - number of launched missiles or UAVs,
    destroyed - number of destroyed missiles or UAVs,
    source - information source (mainly facebook posts)



We will attempt to probabilistically model this dataset using a Naive Bayes classifier from scratch, not using any premade implementations.

If we are able to predict strike locations, then Ukraine could devise an early warning system that would save civilian lives and protect critical infrastructure.




## Bayesian Classification

Naive Bayes classifiers are built on Bayesian classification methods.
These rely on Bayes's theorem, which is an equation describing the relationship of conditional probabilities of statistical quantities.
In Bayesian classification, we're interested in finding the probability of a label $L$ given some observed features, which we can write as $P(L~|~{\rm features})$.
Bayes's theorem tells us how to express this in terms of quantities we can compute more directly:

$$
P(L~|~{\rm features}) = \frac{P({\rm features}~|~L)P(L)}{P({\rm features})}
$$

If we are trying to decide between two labels—let's call them $L_1$ and $L_2$—then one way to make this decision is to compute the ratio of the posterior probabilities for each label:

$$
\frac{P(L_1~|~{\rm features})}{P(L_2~|~{\rm features})} = \frac{P({\rm features}~|~L_1)}{P({\rm features}~|~L_2)}\frac{P(L_1)}{P(L_2)}
$$
Thus removing the need to compute $P(features)$.


All we need now is some model by which we can compute $P({\rm features}~|~L_i)$ for each label.
Such a model is called a *generative model* because it specifies the hypothetical random process that generates the data.
Specifying this generative model for each label is the main piece of the training of such a Bayesian classifier.
The general version of such a training step is a very difficult task, but we can make it simpler through the use of some simplifying assumptions about the form of this model.

This is where the "naive" in "naive Bayes" comes in: if we make very naive assumptions about the generative model for each label, we can find a rough approximation of the generative model for each class, and then proceed with the Bayesian classification.

## Download Dataset

In [ ]:
!wget https://github.com/SOCOM-Ignite/ML_AI_Kickoff/releases/download/v1.0.0/UkraineMissileAttacks.zip
!unzip -o UkraineMissileAttacks.zip -d data/

## Import Libraries

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

## Preparing Data

In [ ]:
### Data Import and Join ###
df1 = pd.read_csv("data/missile_attacks_daily.csv")
df2 = pd.read_csv("data/missiles_and_uav.csv")
df = pd.merge(df1, df2, on='model', how='left')

### Converting time_start and time_end to datetime objects ###
df['date_start'] = pd.to_datetime(df['time_start'], errors='coerce')
default_date = pd.to_datetime('2024-03-24')
df['date_start'].fillna(default_date, inplace=True)
df['date_start'] = df['date_start'].astype(str)

### Data Cleanup ###
def normalize_strings(sample):
    sample = str(sample).replace('and', ',')
    sample = [location.lstrip().rstrip() for location in sample.split(',')]
    sample = ','.join(sample)
    return sample

df['target'] = df['target'].map(normalize_strings)
df['model'] = df['model'].map(normalize_strings)
df['launch_place'] = df['launch_place'].map(normalize_strings)

In [ ]:
df.head()

In [ ]:
### Encode features as binary vectors ###
def binarize_column(df, column_name):
    unique_values = df[column_name].unique()
    new_columns = []
    for value in unique_values:
        new_columns.extend(value.split(','))

    new_columns = set(f"{column_name}_{value}" for value in new_columns if value not in ('', 'nan'))

    new_columns = pd.DataFrame(0, columns=list(new_columns), index=df.index)

    for i, element in enumerate(df[column_name]):
        for x in element.split(','):
            if x not in ('', 'nan'):
                new_columns.at[i, f"{column_name}_{x}"] = 1



    return new_columns

We will attempt to predict the target location of strikes given launch location, types of drones/missiles involved, and number launched.
$$
P(\text{target location}~|~{\rm \text{launch location, drone/missile model, number launched}})$$

In [ ]:
dataset = pd.concat([binarize_column(df, 'target'), binarize_column(df, 'model'), binarize_column(df, 'launch_place'), df['launched']], axis=1)
dataset.head()

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
def get_labels(dataset, column_selector):
    return dataset[[col for col in dataset.columns if column_selector(col)]]

In [ ]:
y_train = get_labels(train_dataset, lambda col: col.startswith('target_'))

### Calculating the prior
To implement Naive Bayes, we need to compute the apriori class probabilities, $P(\text{target location})$. While we could make the prior a uniform distribution, that would be uninformative. Here, we can compute a prior given our training labels.

$$\text{prior for a given class} = \frac{\text{no. of samples in that class}}{\text{total no. of samples}}$$

Using **y_train**, compute the prior.

In [ ]:
prior = # TODO: Implement the calculation for the prior

In [ ]:
### Make sure this cell passes before moving on!
assert (prior.index == y_train.columns).all()
assert prior.sum() == 1

For computing the likelihood $P(\text{launched}~|~\text{target location})$, we will assume the feature is a Gaussian distribution. The calculation for the Gaussian likelihood is:
$$P(x_i~|~y) = \frac{1}{\sqrt{2\pi\sigma^2_y}}\exp \Bigg( -\frac{(x_i-\mu_y)^2}{2\sigma^2_y} \Bigg) $$

In [ ]:
def calc_gaussian_likelihood_params(dataset, column):
    likelihood_params = pd.DataFrame(columns=y_train.columns, index=['mean', 'std'], dtype=float)
    for col in y_train.columns:
        likelihood_params.loc['mean', col] = dataset.loc[dataset[col] == 1, column].mean()
        likelihood_params.loc['std', col] = dataset.loc[dataset[col] == 1, column].std()

    stds = likelihood_params.loc['std']
    std_min = likelihood_params.loc['std', stds > 0].min()
    likelihood_params.loc['std'] = likelihood_params.loc['std'].fillna(std_min)
    likelihood_params.loc['std', stds < std_min] = std_min
    return likelihood_params

In [ ]:
def calc_gaussian_likelihood(likelihood_params, x):
    # expects x to have shape [1, c]
    x = x.transpose()
    mu = likelihood_params.loc[['mean']].to_numpy()
    sigma = likelihood_params.loc[['std']].to_numpy()
    likelihood = 1 / np.sqrt(2 * np.pi * (sigma**2)) * np.exp(-1 * (x - mu)**2 / (2 * (sigma**2)))
    return pd.DataFrame(data=likelihood, columns=likelihood_params.columns)

In [ ]:
likelihood_params_launched = calc_gaussian_likelihood_params(train_dataset, 'launched')

In [ ]:
# testing
calc_gaussian_likelihood(likelihood_params_launched, np.array([[1,2,3]]))

For the model type $P(\text{model}~|~\text{target})$ and launch location$P(\text{launch_location}~|~\text{target})$, we will use the Bernoulli Naive Bayes likelihood. The equation is
$$P(x_i~|~y) = P(x_i=1|y)x_i + (1-P(x_i=1|y))(1-x_i)$$

In [ ]:
def calc_bernoulli_likelihood_params(dataset, column_selector):

    likelihood_params = pd.DataFrame(
        index=[col for col in dataset.columns if column_selector(col)],
        columns=y_train.columns,
        dtype=float
    )

    for col in likelihood_params.columns:
        likelihood_params.loc[:, col] = dataset.loc[dataset[col] == 1, likelihood_params.index].mean(axis=0)

    likelihood_params = likelihood_params.fillna(0)
    return likelihood_params

In [ ]:
def calc_bernoulli_likelihood(likelihood_params, x):
    x_ = x.to_numpy().transpose()
    likelihood_params_ = likelihood_params.to_numpy() # cxt
    bernoulli_likelihood = np.multiply(x_, likelihood_params_) + np.multiply(1 - x_, 1 - likelihood_params_)
    return pd.DataFrame(data=bernoulli_likelihood,
                        columns=likelihood_params.columns,
                        index=likelihood_params.index)

In [ ]:
likelihood_params_model = calc_bernoulli_likelihood_params(train_dataset, lambda col: col.startswith('model_'))
likelihood_params_launch_place = calc_bernoulli_likelihood_params(train_dataset, lambda col: col.startswith('launch_place_'))

In [ ]:
# testing
calc_bernoulli_likelihood(likelihood_params_model, train_dataset.loc[[0], likelihood_params_model.index])

In [ ]:
X_train = get_labels(train_dataset, lambda col: not col.startswith('target_'))
y_test = get_labels(test_dataset, lambda col: col.startswith('target_'))
X_test = get_labels(test_dataset, lambda col: not col.startswith('target_'))

So far, we have constructed a Bayes probability model. To turn this model into a classifier, we pair it with a decision rule. One common rule is to pick the hypothesis that is most probable so as to minimize the probability of misclassification; this is known as the *maximum a posteriori* or MAP decision rule. The corresponding classifier, a Bayes classifier, is the function that assigns a class label
$\hat{y} = C_k$ for some $k$:
$$
\hat{y} = \text{argmax}_{k\in\{1,\ldots,K\}}P(C_k)∏^n_{i=1}P(x_i~|~C_k)
$$

Since the target location is multilabel (the strike can hit multiple locations), we will return the top-k hypotheses from our classifier.

In [ ]:
# Try playing around with the topk parameter!
def make_prediction(sample, topk=3):
    launch_place = sample.loc[[col for col in sample.index if col.startswith('launch_place_')]]
    model = sample.loc[[col for col in sample.index if col.startswith('model_')]]
    launched = sample['launched']

    likelihood_launch_place = calc_bernoulli_likelihood(likelihood_params_launch_place, launch_place.to_frame().transpose())
    likelihood_model = calc_bernoulli_likelihood(likelihood_params_model, model.to_frame().transpose())
    likelihood_launched = calc_gaussian_likelihood(likelihood_params_launched, np.array([launched]))

    likelihood = likelihood_launch_place.prod() * likelihood_model.prod() * likelihood_launched
    # not actually the posterior probability, proportional to it
    posterior = likelihood * prior
    posterior = posterior.to_numpy().squeeze()
    ind = np.argpartition(posterior, -topk)[-topk:]

    prediction = np.zeros_like(posterior)
    prediction[ind] = 1
    return prediction


We will use recall as our metric of choice. In the context of defending against missile strikes, recall gives us the number of strike locations we predicted correctly divided by the total number of strike locations.

Logically, false negatives in this context are far more concerning and deadly than false positives.

In [ ]:
metrics = []
for (_, x), (_, y) in zip(X_test.iterrows(), y_test.iterrows()):
    y_pred = make_prediction(x)
    metrics.append(recall_score(y.to_numpy(), y_pred, average='binary', zero_division=1))

print("Test set recall:", np.mean(metrics))

# Challenge 1
Please also compute the precision score for our classifier.
# Challenge 2
Compute the baseline recall and precision if we were to simply use only the prior.
# Challenge 3
Another model that could be useful to Ukrainian operators would be if we could predict the type(s) of drones and missiles sent in an attack.
Modify the notebook to compute $P(\text{model}~|~\text{target location, launch location, number of launches})$.